In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import pickle
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,f1_score
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_fa5a41007af545d79345a182de0d01bf = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='_xeqXPjOA9vfFqWiNsI5jjKWDuvCHs_XEyEgV-50I8ZP',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

body = client_fa5a41007af545d79345a182de0d01bf.get_object(Bucket='hackathonhomeloan-donotdelete-pr-przadhoxxs4grp',Key='home_loan_approval.xlsx')['Body']

data = pd.read_excel(body.read())



In [3]:
data.columns

Index(['phone', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'Applicant_Income', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [4]:
data.isnull().any()

phone                False
Married              False
Dependents           False
Education            False
Self_Employed         True
Applicant_Income     False
CoapplicantIncome    False
LoanAmount            True
Loan_Amount_Term      True
Credit_History        True
Property_Area        False
Loan_Status          False
dtype: bool

In [5]:
data['Self_Employed'].fillna(data['Self_Employed'].mode()[0],inplace=True)
data['LoanAmount'].fillna(data['LoanAmount'].median(),inplace=True)
data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].median(),inplace=True)
data['Credit_History'].fillna(data['Credit_History'].mode()[0],inplace=True)

In [6]:
data

,phone,Married,Dependents,Education,Self_Employed,Applicant_Income,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,9456732610,No,0,Graduate,No,5000,0,118.0,360.0,1.0,Urban,Y
1,9456798085,Yes,1,Graduate,No,6000,1508,128.0,360.0,1.0,Rural,N
2,9456774514,Yes,0,Graduate,Yes,10000,0,66.0,360.0,1.0,Urban,Y
3,9456761419,Yes,0,Not Graduate,No,10000,2358,120.0,360.0,1.0,Urban,Y
4,7694425773,No,0,Graduate,No,12500,0,141.0,360.0,1.0,Urban,Y
5,9456764038,Yes,2,Graduate,Yes,14000,4196,267.0,360.0,1.0,Urban,Y
6,7694428921,Yes,0,Not Graduate,No,15000,1516,95.0,360.0,1.0,Urban,Y
7,9456722134,Yes,3+,Graduate,No,18000,2504,158.0,360.0,0.0,Semiurban,N
8,9456748324,Yes,2,Graduate,No,19000,1526,168.0,360.0,1.0,Urban,Y
9,9456756181,Yes,1,Graduate,No,20000,10968,349.0,360.0,1.0,Semiurban,N


In [7]:
data["Dependents"] = data["Dependents"].astype("string")

In [8]:
data = data.drop(['phone'],axis=1)

In [9]:
data=data.drop(['Married','Self_Employed'],axis=1)
data

,Dependents,Education,Applicant_Income,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,Graduate,5000,0,118.0,360.0,1.0,Urban,Y
1,1,Graduate,6000,1508,128.0,360.0,1.0,Rural,N
2,0,Graduate,10000,0,66.0,360.0,1.0,Urban,Y
3,0,Not Graduate,10000,2358,120.0,360.0,1.0,Urban,Y
4,0,Graduate,12500,0,141.0,360.0,1.0,Urban,Y
5,2,Graduate,14000,4196,267.0,360.0,1.0,Urban,Y
6,0,Not Graduate,15000,1516,95.0,360.0,1.0,Urban,Y
7,3+,Graduate,18000,2504,158.0,360.0,0.0,Semiurban,N
8,2,Graduate,19000,1526,168.0,360.0,1.0,Urban,Y
9,1,Graduate,20000,10968,349.0,360.0,1.0,Semiurban,N


In [10]:
#label encoding
le = LabelEncoder()
data['Education'] = le.fit_transform(data['Education'])
data['Loan_Status'] = le.fit_transform(data['Loan_Status'])
data['Property_Area'] = le.fit_transform(data['Property_Area'])
data['Dependents'] = le.fit_transform(data['Dependents'])

In [11]:
x1 = data.iloc[:,:8]
y1 = data.iloc[:,8:]

In [12]:
# divide data
x = data.iloc[:,:8].values
y = data.iloc[:,8:].values

In [13]:
data

,Dependents,Education,Applicant_Income,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,0,5000,0,118.0,360.0,1.0,2,1
1,1,0,6000,1508,128.0,360.0,1.0,0,0
2,0,0,10000,0,66.0,360.0,1.0,2,1
3,0,1,10000,2358,120.0,360.0,1.0,2,1
4,0,0,12500,0,141.0,360.0,1.0,2,1
5,2,0,14000,4196,267.0,360.0,1.0,2,1
6,0,1,15000,1516,95.0,360.0,1.0,2,1
7,3,0,18000,2504,158.0,360.0,0.0,1,0
8,2,0,19000,1526,168.0,360.0,1.0,2,1
9,1,0,20000,10968,349.0,360.0,1.0,1,0


In [14]:
#one hot encoding for Property Area and Dependents
one = OneHotEncoder()
dependents = one.fit_transform(x[:,:1]).toarray()
propertyy = one.fit_transform(x[:,7:8]).toarray()

In [15]:
x1 = x1.drop(['Dependents','Property_Area'],axis=1)

In [16]:
x = np.delete(x,0,axis=1)

In [17]:
x = np.delete(x,6,axis=1)

In [18]:
dependents

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [19]:
dependants1 = pd.DataFrame(dependents, columns = ['Dependents1','Dependents2','Dependents3','Dependents4'])

In [20]:
x1 = pd.concat([x1, dependants1], axis=1)

In [21]:
x = np.concatenate((x,dependents),axis=1)

In [22]:
property1 = pd.DataFrame(propertyy, columns = ['property1','property2','property3'])

In [23]:
x1 = pd.concat([x1, property1], axis=1)

In [24]:
x = np.concatenate((x,propertyy),axis=1)
x.shape

(40, 13)

In [25]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [26]:
X_train,X_test,Y_train,Y_test = train_test_split(x1,y1,test_size=0.2,random_state=0)

In [27]:
#standard scaling because classification
#sc =  StandardScaler()
#x_train = sc.fit_transform(x_train)
#x_test = sc.transform(x_test)


In [28]:
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

In [29]:
X_train = pd.DataFrame(X_train, columns = ['Education','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Dependents1','Dependents2','Dependents3','Dependents4','property1','property2','property3'])

In [30]:
#Decision Tree
def decisionTree(x_train,x_test,y_train,y_test):
    dtc = DecisionTreeClassifier(criterion='entropy',random_state=0)
    dtc.fit(x_train,y_train)
    y_pred_dtc = dtc.predict(x_test)
    print("Decision Tree Classifer\n")
    print("Confusion Matrix")
    print(confusion_matrix(y_test,y_pred_dtc))
    print("Classification Report")
    print(classification_report(y_test,y_pred_dtc))
    print("Accuracy: ")
    print(accuracy_score(y_test,y_pred_dtc))

In [31]:
#Random Forest Classifier
def randomForest(x_train,x_test,y_train,y_test):
    rfc=RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=0)
    rfc.fit(x_train,y_train)
    y_pred_rfc = rfc.predict(x_test)
    print("Random Forest Classifer\n")
    print("Confusion Matrix")
    print(confusion_matrix(y_test,y_pred_rfc))
    print("Classification Report")
    print(classification_report(y_test,y_pred_rfc))
    print("Accuracy: ")
    print(accuracy_score(y_test,y_pred_rfc))

In [32]:
#KNeighbours Classifier
def KNN(x_train,x_test,y_train,y_test):
    knn = KNeighborsClassifier(n_neighbors=5,metric='minkowski')
    knn.fit(x_train,y_train)
    y_pred_knn = knn.predict(x_test)
    print("KNeighbours Classifer\n")
    print("Confusion Matrix")
    print(confusion_matrix(y_test,y_pred_knn))
    print("Classification Report")
    print(classification_report(y_test,y_pred_knn))
    print("Accuracy: ")
    print(accuracy_score(y_test,y_pred_knn))

In [33]:
#XGBoost Classifier
def xgboost(x_train,x_test,y_train,y_test):
    #tree = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
    #xg=xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric='mlogloss')
    #xg.fit(x_train,y_train.ravel())
    lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]
    print("Gradient Boost Classifer\n")
    for learning_rate in lr_list:
        gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
        gb_clf.fit(x_train, y_train)

        print("Learning rate: ", learning_rate)
        print("Accuracy score (training): {0:.3f}".format(gb_clf.score(x_train, y_train)))
        print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(x_test, y_test)))
    
   

In [34]:
#Comparing the models
randomForest(x_train,x_test,y_train,y_test)

Random Forest Classifer

Confusion Matrix
[[1 2]
 [0 5]]
Classification Report
              precision    recall  f1-score   support

           0       1.00      0.33      0.50         3
           1       0.71      1.00      0.83         5

    accuracy                           0.75         8
   macro avg       0.86      0.67      0.67         8
weighted avg       0.82      0.75      0.71         8

Accuracy: 
0.75


/tmp/wsuser/ipykernel_164/1211179674.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(x_train,y_train)


In [35]:
decisionTree(x_train,x_test,y_train,y_test)

Decision Tree Classifer

Confusion Matrix
[[3 0]
 [1 4]]
Classification Report
              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      0.80      0.89         5

    accuracy                           0.88         8
   macro avg       0.88      0.90      0.87         8
weighted avg       0.91      0.88      0.88         8

Accuracy: 
0.875


In [36]:
KNN(x_train,x_test,y_train,y_test)

KNeighbours Classifer

Confusion Matrix
[[1 2]
 [3 2]]
Classification Report
              precision    recall  f1-score   support

           0       0.25      0.33      0.29         3
           1       0.50      0.40      0.44         5

    accuracy                           0.38         8
   macro avg       0.38      0.37      0.37         8
weighted avg       0.41      0.38      0.38         8

Accuracy: 
0.375


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [37]:
xgboost(x_train,x_test,y_train,y_test)

Gradient Boost Classifer

Learning rate:  0.05
Accuracy score (training): 0.906
Accuracy score (validation): 0.875
Learning rate:  0.075
Accuracy score (training): 0.938
Accuracy score (validation): 0.875
Learning rate:  0.1
Accuracy score (training): 0.938
Accuracy score (validation): 0.875
Learning rate:  0.25
Accuracy score (training): 0.969
Accuracy score (validation): 0.875
Learning rate:  0.5
Accuracy score (training): 1.000
Accuracy score (validation): 0.875
Learning rate:  0.75
Accuracy score (training): 1.000
Accuracy score (validation): 1.000
Learning rate:  1
Accuracy score (training): 1.000
Accuracy score (validation): 1.000


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

In [38]:
#since xgboost has the highest accuracy we are going to export that model to pickle
#xg=xgb.XGBClassifier(objective="binary:logistic", random_state=42)
#xg.fit(x_train,y_train)
#y_pred_xg=xg.predict(x_test)


dt_clf = DecisionTreeClassifier(criterion='entropy',random_state=0)
dt_clf.fit(x_train, y_train)
y_pred = dt_clf.predict(x_test)

In [39]:
print(accuracy_score(y_test,y_pred))

0.875


In [40]:
!pip install -U ibm-watson-machine-learning

     |████████████████████████████████| 1.8 MB 20.6 MB/s eta 0:00:01
  Attempting uninstall: ibm-watson-machine-learning
    Found existing installation: ibm-watson-machine-learning 1.0.232
    Uninstalling ibm-watson-machine-learning-1.0.232:
      Successfully uninstalled ibm-watson-machine-learning-1.0.232


In [41]:
from ibm_watson_machine_learning import APIClient
import json
import numpy as np

In [42]:
wml_credentials={
    "name": "hackathon_home_loan",
    "description": "hackathon_home_loan",
    "createdAt": "2022-09-07T09:37+0000",
    "apikey": "hy7E-bnXomI-gSe1K6KlfCdqIp5KFD2BWiLdAbVqnnEC",
    "url": "https://us-south.ml.cloud.ibm.com"
}

In [43]:
wml_client = APIClient(wml_credentials)
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -----------------------  ------------------------
ID                                    NAME                     CREATED
54931202-cd2e-4cbe-a7c2-2866f668bc72  hackathon-home-loan      2022-09-07T09:43:53.583Z
3e34308a-4a80-4145-b009-7637e8e2963f  Loan Approval Predictor  2022-05-30T17:47:46.756Z
6d456318-2c91-4ca5-a443-b19563e1e7f7  Startup_Predictor        2022-05-22T11:49:45.681Z
------------------------------------  -----------------------  ------------------------


In [44]:
SPACE_ID = "3e34308a-4a80-4145-b009-7637e8e2963f"
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [45]:
software_spec_uid = wml_client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [46]:
MODEL_NAME = 'hackathon home loan'
DEPLOYMENT_NAME = 'Sklearn Deployment'
BEST_MODEL = dt_clf

In [47]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

In [48]:
model_details = wml_client.repository.store_model(model=BEST_MODEL, meta_props=model_props, training_data=None)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1452: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [49]:
model_id = wml_client.repository.get_model_uid(model_details)
model_id

This method is deprecated, please use get_model_id()


'58cc102d-ea18-443b-bacd-267980d4e864'